In [1]:
import numpy as np
import pandas as pd
import pyodbc
import pandas as pd
import numpy as np
from pprint import pprint
import datetime
from configparser import ConfigParser
import datetime
import pandas as pd
import sqlalchemy as sa

In [2]:
from configparser import ConfigParser
import sqlalchemy
import pyodbc
import urllib
from urllib.parse import quote
from abc import ABC
import logging
import sys
import re
import pandas as pd
import numpy as np
config = ConfigParser()
config.read('fingate_to_bridge_config.ini')
#config.read('/notebooks/notebooks/fingate_to_bridge_config.ini')
config_data=config['SERVER']
import warnings
warnings.filterwarnings('ignore')

In [3]:
conn = pyodbc.connect(config_data["fincore_bridge_read"])
cursor = conn.cursor()

In [4]:
col = pd.read_sql_query('''Select top 1 * from CBDT_DataLoad.CBDT.[Cbdt_Master_Data] with (nolock) ''',conn)
col_df = col.columns.to_list()

In [5]:
with open('PAN_data1_by') as file:
    data = file.read().splitlines()

In [6]:
d = pd.DataFrame(data,columns = ['DATA'])

In [7]:
d['COUNT'] = d['DATA'].apply(lambda x : x.count('~'))

In [8]:
d

,DATA,COUNT
0,"""BJMPK5453K""~""HARVINDRA KOUR""~""MEHARBAAN SIN...",8
1,"""AYTPJ8100B""~""SHITAL JANA""~""KALI GHOSH""~""F""~...",8
2,"""AJQPK3960N""~""CHINTAMANI SAGARU KHUTARKAR""~""SA...",8
3,"""AAHPD6569L""~""VINCENT DAVID""~""JOSEPH DAVID""~...",8
4,"""APNPP0567K""~""SUBODH PANDEY""~""UMESH CHANDRA P...",8
...,...,...
4999995,"""APHPR0220D""~""RAJENDRA SINGH RUNDLA""~""RAMESHWA...",8
4999996,"""AWQPM3393N""~""SAMASER MAHAMMAD""~""SAUKAT MAHA...",8
4999997,"""AMOPD2190G""~""SUDHA SHYAMKANT DUKALE""~""DATTATR...",8
4999998,"""BTJPV1822L""~""MANDODARI SITARAM VAYAL""~""NAMDEV...",8


In [9]:
d[d['COUNT'] != 8 ]

,DATA,COUNT


In [10]:
if (len(d[d['COUNT']!= 8]) != 0 ):
    raise SystemExit


In [11]:
col = pd.read_sql_query('''Select top 1 * from CBDT_DataLoad.CBDT.[Cbdt_Master_Data] with (nolock) ''',conn)
col_df = col.columns.to_list()

In [12]:
col_df.remove('Create_Date')
col_df.remove('Create_By')
col_df.remove('File_Name')

In [13]:
print(str(datetime.datetime.now()))

2023-08-24 15:51:37.939724


In [14]:
data = pd.read_csv(r'PAN_data1_by',sep = '~',names = col_df)

In [15]:
data['File_Name'] = 'PAN_data1_by'

In [16]:
data['Create_Date'] = str(datetime.datetime.now())

In [17]:
data['Create_By'] = 'HDM'

In [25]:
config = ConfigParser()
config.read(r'fingate_to_bridge_config.ini')
connection_string = config.get('SERVER', 'fincore_bridge_read')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

connection_url = sa.engine.URL.create(
    "mssql+pyodbc", 
    query=dict(odbc_connect=connection_string)
)
engine = sa.create_engine(connection_url, fast_executemany=True)



In [26]:
# # Deleting existing data in SQL Table:-
# with engine.begin() as conn:
#     conn.exec_driver_sql("DELETE FROM CBDT_DataLoad.CBDT.[Cbdt_Master_Data] ")

In [27]:
df1 = data[:2500000]
df2 = data[2500000:4000000]
df3 = data[4000000:]

In [29]:
# upload the DataFrame
df1.to_sql("Cbdt_Master_Data", engine, schema="CBDT_DataLoad.CBDT", if_exists="append", index=False,chunksize=1000)

DBAPIError: (pyodbc.Error) ('HY000', '[HY000] [Microsoft][ODBC SQL Server Driver]Unknown token received from SQL Server (0) (SQLExecute)')
[SQL: INSERT INTO [CBDT_DataLoad].[CBDT].[Cbdt_Master_Data] ([Pan], [Entity_Name], [Father_Name], [Gender], [Dob_Or_Doi], [Residential_Address], [Communication_Address], [Email_Id], [Telephone_Number], [PRESENT], [File_Name], [Create_Date], [Create_By]) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: (('CHMPA9698F', 'KANDOOR GANESH AITHAL', 'ANANTHAYYA  AITHAL', 'M', '1957-10-03', '1-93 CHOUKADA PALU HOUSE  MANCHI VILLAGE  MANCHI POST  BANTWAL TALUK  DAKSHINA KANNADA DISTRICT', '1-93 CHOUKADA PALU HOUSE  MANCHI VILLAGE  MANCHI POST  BANTWAL TALUK  DAKSHINA KANNADA DISTRICT', None, None, None, 'PAN_data1_by', '2023-08-24 15:52:20.411312', 'HDM'), ('CRUPG5919H', 'MD  GULZAR', 'MD NASIM UDDIN', 'M', '1969-01-01', 'MD NASIM UDDIN  WARD NO 03  ISMAILPUR  KHAGARIA', 'MD NASIM UDDIN  WARD NO 03  ISMAILPUR  KHAGARIA', None, None, None, 'PAN_dat

In [ ]:
df2.to_sql("Cbdt_Master_Data", engine, schema="CBDT_DataLoad.CBDT", if_exists="append", index=False,chunksize=1000)

In [ ]:
df3.to_sql("Cbdt_Master_Data", engine, schema="CBDT_DataLoad.CBDT", if_exists="append", index=False,chunksize=1000)

In [ ]:
print(str(datetime.datetime.now()))

In [ ]:
cursor.execute('''
update CBDT_DATALOAD.CBDT.[CBDT_MASTER_DATA] set  present = 'Y' from 
CBDT_DATALOAD.CBDT.[CBDT_MASTER_DATA] A (NOLOCK)
inner JOIN 
EXTL_DB.CBDT.[CBDT_MASTER_DATA] B (NOLOCK)
ON A.PAN = B.PAN
''')
conn.commit()

In [ ]:
cursor.execute('''UPDATE CBDT_DATALOAD.CBDT.[CBDT_MASTER_DATA]  SET PRESENT = 'N' WHERE PRESENT IS NULL''')
conn.commit()

In [ ]:
cursor.execute('''
insert into CBDT_DATALOAD.CBDT.CBDT_MASTER_DATA_FINAL
from 
SELECT * FROM CBDT_DATALOAD.CBDT.CBDT_MASTER_DATA
''')
conn.commit()

In [ ]:
cursor.execute('''delete from CBDT_DATALOAD.CBDT.CBDT_MASTER_DATA where PRESENT = 'Y' ''')
conn.commit()

In [ ]:
cursor.execute('''INSERT INTO EXTL_DB.CBDT.CBDT_MASTER_DATA 
SELECT PAN,ENTITY_NAME,FATHER_NAME,GENDER,DOB_OR_DOI,RESIDENTIAL_ADDRESS,COMMUNICATION_ADDRESS,EMAIL_ID,TELEPHONE_NUMBER,FILE_NAME,CREATE_DATE,CREATE_BY
FROM CBDT_DATALOAD.CBDT.CBDT_MASTER_DATA''')
conn.commit()